## 생존 여부 예측하기
### 1. train.csv / test.csv : 타이타닉 탑승자들 중 일부의 인적 정보와 생존 여부 데이터
- PassengerID : 탑승객 고유 아이디
- Survival : 탑승객 생존 유무 (0: 사망, 1: 생존)
- Pclass : 등실의 등급
- Name : 이름
- Sex : 성별
- Age : 나이
- Sibsp : 함께 탐승한 형제자매, 아내, 남편의 수
- Parch : 함께 탐승한 부모, 자식의 수
- Ticket :티켓 번호
- Fare : 티켓의 요금
- Cabin : 객실번호
- Embarked : 배에 탑승한 항구 이름 ( C = Cherbourn, Q = Queenstown, S = Southampton)  

### 2. sample_submission.csv : 정답 파일의 예시

#### `1.` 라이브러리 임포트

In [1]:
import pandas as pd
import matplotlib.pyplot as plt                     # 데이터 전처리 과정 중 시각화를 할때 사용합니다.
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier     # 의사 결정 나무 알고리즘을 임포트합니다.
from sklearn.model_selection import cross_val_score # 모델의 성능을 평가하기 위한 임포트입니다.

#### `2.` 데이터 로드

In [2]:
train_data = pd.read_csv('./타이타닉/train.csv')
test_data = pd.read_csv('./타이타닉/test.csv')
submission_data = pd.read_csv('./타이타닉/submission.csv')

#### `3.` 데이터 정보 확인

In [3]:
display(train_data.head())
print('---------------train_data.info()---------------')
print(train_data.info())

display(test_data.head())
print('---------------test_data.info()---------------')
print(test_data.info())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


---------------train_data.info()---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


---------------test_data.info()---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


#### `4.` 데이터 전처리: 결측값 처리

In [4]:
print('-----수정 전 train_data.isnull().sum()-----')
print(train_data.isnull().sum())
print('-----수정 전 test_data.isnull().sum()-----')
print(test_data.isnull().sum())
print()

# Age의 NaN값을 Age의 평균 값으로 대체합니다.
train_data['Age'] = train_data['Age'].fillna(np.mean(train_data['Age']))
test_data['Age'] = test_data['Age'].fillna(np.mean(test_data['Age']))

# test_data의 Fare 결측값 1개를 평균 값으로 대체합니다.
test_data['Fare'] = test_data['Fare'].fillna(np.mean(test_data['Fare']))

# 타깃값과의 연관성이 적어 보이므로 train_data['Embarked'] 결측값 2개를 고유값 3개중 가장 많은 'S'로 대체합니다.
print('-----클래스별 데이터 개수 확인-----')
print(train_data['Embarked'].value_counts())
train_data['Embarked'] = train_data['Embarked'].fillna('S')
print()

print('-----수정 후 train_data.isnull().sum()-----')
print(train_data.isnull().sum())
print('-----수정 후 test_data.isnull().sum()-----')
print(test_data.isnull().sum())

-----수정 전 train_data.isnull().sum()-----
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
-----수정 전 test_data.isnull().sum()-----
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

-----클래스별 데이터 개수 확인-----
S    644
C    168
Q     77
Name: Embarked, dtype: int64

-----수정 후 train_data.isnull().sum()-----
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64
-----수정 후 test_data.isnull().sum()-----
PassengerId      0
Pclass           0
Name         

#### `5.` 데이터 전처리: 드랍, 변환

In [5]:
# 먼저 Cabin은 결측값이 전체 데이터(학습, 테스트)의 약 77%이므로 대체하기 어렵다고 판단하여 특성 전체를 드랍하겠습니다.
train_data.drop(columns=['Cabin'], inplace=True)
test_data.drop(columns=['Cabin'], inplace=True)

# Name은 생존율과 관련이 없다고 판단되므로 전체 드랍하겠습니다.
train_data.drop(columns=['Name'], inplace=True)
test_data.drop(columns=['Name'], inplace=True)

# PassengerId도 생존율과 관련이 없다고 판단됩니다.
train_data.drop(columns=['PassengerId'], inplace=True)
test_data.drop(columns=['PassengerId'], inplace=True)

# Ticket의 형태를 변환합니다. object → int
# Ticket열을 문자열과 정수형으로 나누고 각 리스트별 뒤에서 첫번째 자리(정수형)의 값을 가져옵니다.
#      x = []
#      for i in train_data['Ticket'] :
#          x.append(i.split()[-1])
# 다음은 한줄로 작성한 코드입니다.
train_data['Ticket'] = [i.split()[-1] for i in list(train_data['Ticket'])]
test_data['Ticket'] = [i.split()[-1] for i in list(test_data['Ticket'])]

# 문자열로만 이루어진 LINE 값이 있는 행을 삭제합니다. Ticket컬럼에 LINE과 같은 행 인덱스를 추출합니다.
con = train_data[train_data['Ticket'] == 'LINE'].index

# 위 코드에서 추출한 행 인덱스 넘버를 삭제하고 원본 데이터에 바로 적용합니다.
train_data.drop(con, inplace=True)

#### `6.` 데이터 전처리: 인코딩

In [6]:
# LabelEncoder 모델을 encoder 변수에 할당합니다.
encoder = LabelEncoder()

# Sex 특성을 인코딩 합니다.
train_data['Sex'] = encoder.fit_transform(train_data['Sex'])
test_data['Sex'] = encoder.fit_transform(test_data['Sex'])

# Embarked 특성을 인코딩 합니다.
train_data['Embarked'] = encoder.fit_transform(train_data['Embarked'])
test_data['Embarked'] = encoder.fit_transform(test_data['Embarked'])

#### `7.` 모델링을 위한 데이터 조정

In [7]:
# 특성값으로 쓸 데이터는 원본 데이터에서 Survived열을 드랍하고 건네줍니다.
feature_train = train_data.drop(['Survived'], axis=1)

# 타깃값으로 쓸 데이터는 원본 데이터에서 Survived열을 건네줍니다.
targit_train = train_data['Survived']

# 중간 확인으로 학습에 이용할 데이터와 테스트 데이터가 같은지 확인합니다.
display(feature_train)
display(test_data)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,22.000000,1,0,21171,7.2500,2
1,1,0,38.000000,1,0,17599,71.2833,0
2,3,0,26.000000,0,0,3101282,7.9250,2
3,1,0,35.000000,1,0,113803,53.1000,2
4,3,1,35.000000,0,0,373450,8.0500,2
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,211536,13.0000,2
887,1,0,19.000000,0,0,112053,30.0000,2
888,3,0,29.699118,1,2,6607,23.4500,2
889,1,1,26.000000,0,0,111369,30.0000,0


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,34.50000,0,0,330911,7.8292,1
1,3,0,47.00000,1,0,363272,7.0000,2
2,2,1,62.00000,0,0,240276,9.6875,1
3,3,1,27.00000,0,0,315154,8.6625,2
4,3,0,22.00000,1,1,3101298,12.2875,2
...,...,...,...,...,...,...,...,...
413,3,1,30.27259,0,0,3236,8.0500,2
414,1,0,39.00000,0,0,17758,108.9000,0
415,3,1,38.50000,0,0,3101262,7.2500,2
416,3,1,30.27259,0,0,359309,8.0500,2


#### `8.` 모델 생성 및 훈련, 평가

In [8]:
# 의사결정나무 모델을 변수 model에 할당합니다.
model = DecisionTreeClassifier()

# 전처리와 조정을 거친 데이터로 모델을 학습시킵니다.
model.fit(feature_train, targit_train)

# 모델의 성능을 평가합니다.
# cross_val_score: 텐서플로우에서 제공하는 기능으로 다양한 평가지표를 기반으로 교차 검증을 진행합니다.
# scoring=: 평가지표로는 생존확률의 예측이니 정확도가 중요하다고 판단하여 정확도를 평가지표로 사용했습니다.
# cv=: 데이터의 폴더 개수를 지정합니다 기본값은 3개 입니다.
model_score = cross_val_score(model, feature_train, targit_train, scoring='accuracy', cv=5)
print('모델의 정확도: ', model_score.mean())

# 예측을 진행합니다.
model_pred = model.predict(test_data)

모델의 정확도:  0.7779724496921221


#### `9.` 제출 형식에 맞게 변환

In [9]:
submission_data['Survived'] = model_pred
submission_data.to_csv('submission.csv', index=False)